#Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import Statements

In [ ]:
!pip install transformers datasets evaluate pandas scikit-learn rouge_score sentence-transformers nltk


In [ ]:
import re
import random
import numpy as np
import pandas as pd
import os
import nltk
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from random import sample

from sentence_transformers import SentenceTransformer

from datasets import Dataset, DatasetDict

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)

import torch
import evaluate
from evaluate import load

from IPython.display import display
from tqdm import tqdm

#Data Preproccessing

In [ ]:
#Load in movies data and critics reviews data. These are 2 separate dataset that need to be joined together

movie_data = "/content/drive/My Drive/SEIS 767 Final Project Dataset/rotten_tomatoes_movies.csv"
critics_reviews_data = "/content/drive/My Drive/SEIS 767 Final Project Dataset/rotten_tomatoes_critic_reviews.csv"

movies = pd.read_csv(movie_data)
critics_reviews = pd.read_csv(critics_reviews_data)

#Join 2 datasets based on the rotten_tomatoes_link. This will give us all the reviews for each movies and their review_score and critics_consensus

movies_with_critics_reviews = pd.merge(movies, critics_reviews, on='rotten_tomatoes_link', how='inner')
reviews = movies_with_critics_reviews[['movie_title', 'review_content', 'review_score','critics_consensus']]

In [ ]:
#Example showing percy_jackson_reviews. 150 total reviews
percy_jackson_reviews = reviews[reviews['movie_title'] == 'Percy Jackson & the Olympians: The Lightning Thief']

percy_jackson_reviews

,movie_title,review_content,review_score,critics_consensus
0,Percy Jackson & the Olympians: The Lightning T...,A fantasy adventure that fuses Greek mythology...,NaN,Though it may seem like just another Harry Pot...
1,Percy Jackson & the Olympians: The Lightning T...,"Uma Thurman as Medusa, the gorgon with a coiff...",NaN,Though it may seem like just another Harry Pot...
2,Percy Jackson & the Olympians: The Lightning T...,With a top-notch cast and dazzling special eff...,NaN,Though it may seem like just another Harry Pot...
3,Percy Jackson & the Olympians: The Lightning T...,Whether audiences will get behind The Lightnin...,3.5/5,Though it may seem like just another Harry Pot...
4,Percy Jackson & the Olympians: The Lightning T...,What's really lacking in The Lightning Thief i...,NaN,Though it may seem like just another Harry Pot...
...,...,...,...,...
144,Percy Jackson & the Olympians: The Lightning T...,The plot uses the presence of the Greek gods a...,2/4,Though it may seem like just another Harry Pot...
145,Percy Jackson & the Olympians: The Lightning T...,As a former middle school teacher of gifted li...,3/4,Though it may seem like just another Harry Pot...
146,Percy Jackson & the Olympians: The Lightning T...,The Lightning Thief is an admirable kid's fant...,NaN,Though it may seem like just another Harry Pot...
147,Percy Jackson & the Olympians: The Lightning T...,"OK, but only just.",2.5/5,Though it may seem like just another Harry Pot...


In [ ]:
# Get the unique values in the 'review_score' column so we can see what needs be to cleaned up
unique_review_scores = reviews['review_score'].unique()
unique_review_scores

array([nan, '3.5/5', '1/4', 'B', '3/5', '4/5', '2/4', '2/5', 'C', '2.5/4',
       '3/4', 'C-', '2.75/5', 'B-', '2.5/5', '5.5/10', '1/5', 'C+', 'D+',
       '1.5/4', '6.5/10', '3/10', '1.5/5', '4/10', '7/10', '6/10', 'B+',
       '5/5', 'A-', '4.5/5', '5.78/10', '3.5/4', 'A', '9/10', '8.5/10',
       '4/4', '83/100', '8/10', '92/100', '82/100', '84/100', '10/10',
       'D-', 'F', '0.5/4', 'D', '5/10', '2/10', '2/6', '0.5/5', '0/4',
       '0/5', '76/100', '87/100', '74/100', '86/100', '81/100', '56/100',
       '78/100', '2.25/5', '8.8/10', '0/10', '44/100', '2.75/4', '2.4/5',
       '8.6/10', '79/100', '68/100', '80/100', '90/100', '66/100',
       '91/100', '0.3/5', '9.2/10', '2.7/5', '57/100', '4/6', '3/6',
       '73/100', '42/100', '65/100', '7.5/10', '4.5/10', '5/6', '1/6',
       '70/100', '3.5/10', '71/100', '0.1/5', '3.25/10', '89/100',
       '40/100', '64/100', '62/100', '1/10', '3.75/5', '37/100', '49/100',
       '60/100', '3.3/5', '77/100', '5.25/10', '1.4/4', '75/100', '

In [ ]:
#Function created with the help of ChatGPT and our list of unique review scores to clean up the different formats of reviews scores

# Define the normalization function
def normalize_score(score):
    # Handle NaN values
    if pd.isna(score):
        return np.nan

    # Handle letter grades (e.g., 'A', 'B', 'C') - define a mapping to numeric scale
    letter_grades = {
        'A': 10, 'A-': 9, 'B+': 8.5, 'B': 8, 'B-': 7, 'C+': 6.5, 'C': 6, 'C-': 5, 'D+': 4.5,
        'D': 4, 'D-': 3, 'F': 0
    }
    if score in letter_grades:
        return round(letter_grades[score], 2)

    # Handle cases with the format 'x/y' where x is the numerator and y is the denominator
    match = re.match(r"(\d+(\.\d+)?)/(\d+(\.\d+)?)", score)
    if match:
        numerator = float(match.group(1))
        denominator = float(match.group(3))
        # Check if the denominator is 0 before dividing
        if denominator == 0:
            return np.nan
        else:
            return round((numerator / denominator) * 10, 2)  # Normalize to a scale of 10

    # Handle percentages like '83/100'
    match_percentage = re.match(r"(\d+(\.\d+)?)/100", score)
    if match_percentage:
        numerator = float(match.group(1))
        return round(numerator, 2)  # Already on a 0-100 scale

    # Handle other special cases (e.g., decimals)
    try:
        score_float = float(score)
        return round(score_float, 2)  # If it's a decimal score like '5.5'
    except ValueError:
        pass  # If it's not a valid numeric score, we'll return NaN

    return np.nan  # If all else fails, return NaN

# Apply the function to the review scores column
reviews['normalized_score'] = reviews['review_score'].apply(normalize_score)


<ipython-input-71-d0b508ab51f8>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['normalized_score'] = reviews['review_score'].apply(normalize_score)


In [ ]:
#Take a look at our Percy Jackson reviews comparing original score and normalized score

percy_jackson_reviews = reviews[reviews['movie_title'] == 'Percy Jackson & the Olympians: The Lightning Thief']

percy_jackson_reviews[['review_score', 'normalized_score']]

,review_score,normalized_score
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,3.5/5,7.0
4,NaN,NaN
...,...,...
144,2/4,5.0
145,3/4,7.5
146,NaN,NaN
147,2.5/5,5.0


In [ ]:
#Set review score to our new normalized scores and drop the extra column
reviews['review_score'] = reviews['normalized_score']
reviews = reviews.drop(columns=['normalized_score'])

#Drop all rows with any Null values. Including Null review scores. This cuts the data in half
reviews = reviews.dropna()

<ipython-input-73-e445d6e19d74>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['review_score'] = reviews['normalized_score']


In [ ]:
#Now we have all the necessary data we need. movie_titles, review_content (which will be our X), and review_scores and critics_consensus (which will be our Y).
#But we still need to group the reviews by movie_title for use by the summarization model

reviews = reviews.reset_index(drop=True)
display(reviews)

,movie_title,review_content,review_score,critics_consensus
0,Percy Jackson & the Olympians: The Lightning T...,Whether audiences will get behind The Lightnin...,7.0,Though it may seem like just another Harry Pot...
1,Percy Jackson & the Olympians: The Lightning T...,Harry Potter knockoffs don't come more transpa...,2.5,Though it may seem like just another Harry Pot...
2,Percy Jackson & the Olympians: The Lightning T...,"Percy Jackson isn't a great movie, but it's a ...",7.0,Though it may seem like just another Harry Pot...
3,Percy Jackson & the Olympians: The Lightning T...,"Fun, brisk and imaginative",8.0,Though it may seem like just another Harry Pot...
4,Percy Jackson & the Olympians: The Lightning T...,"Crammed with dragons, set-destroying fights an...",6.0,Though it may seem like just another Harry Pot...
...,...,...,...,...
682657,Zulu,The old-fashioned war epic that made Michael C...,7.5,Zulu patiently establishes a cast of colorful ...
682658,Zulu,A brilliantly made dramatisation of Rorke's Dr...,8.5,Zulu patiently establishes a cast of colorful ...
682659,Zulu,The movie is a revelation.,10.0,Zulu patiently establishes a cast of colorful ...
682660,Zulu,"This is that rarity in films - an all-action, ...",10.0,Zulu patiently establishes a cast of colorful ...


In [ ]:
#Exploratory statistics

average_num_reviews = int(round(reviews.groupby('movie_title').size().mean()))

print(f"The average number of reviews per movie title is approximately: {average_num_reviews}")

num_unique_movies = reviews['movie_title'].nunique()
print(f"The number of unique movie titles is: {num_unique_movies}")

reviews['word_count'] = reviews['review_content'].apply(lambda x: len(x.split()))
average_word_count = reviews['word_count'].mean().round(0)
print(f"The average number of words in all reviews is: {average_word_count}")

movie_review_counts = reviews.groupby('movie_title').size()
movie_with_most_reviews = movie_review_counts.idxmax()
max_reviews = movie_review_counts.max()

print(f"The movie with the most reviews is '{movie_with_most_reviews}' with {max_reviews} reviews.")

The average number of reviews per movie title is approximately: 77
The number of unique movie titles is: 8911
The average number of words in all reviews is: 21.0
The movie with the most reviews is 'Star Wars: The Rise of Skywalker' with 672 reviews.


In [ ]:
# Set a max number of reviews
MAX_REVIEWS_PER_MOVIE = 30

# Sample reviews per movie
sampled_reviews = (
    reviews.groupby("movie_title")
    .apply(lambda x: x.sample(n=MAX_REVIEWS_PER_MOVIE, random_state=2025) if len(x) >= MAX_REVIEWS_PER_MOVIE else x)
    .reset_index(drop=True)
)

# Group sampled reviews into a single string per movie
grouped_reviews = sampled_reviews.groupby("movie_title").agg(
    review_content=("review_content", lambda x: " ".join(x)),
    review_score=("review_score", "mean"),
    critics_consensus=("critics_consensus", "first")
).reset_index()

grouped_reviews["review_score"] = grouped_reviews["review_score"].round(1)

print(f"With a max reviews per movie of {MAX_REVIEWS_PER_MOVIE}, the average number of tokens will be approximately: {round(MAX_REVIEWS_PER_MOVIE * average_word_count * 1.3)} to {round(MAX_REVIEWS_PER_MOVIE * average_word_count * 1.5)}")


<ipython-input-76-d241645d5def>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=MAX_REVIEWS_PER_MOVIE, random_state=2025) if len(x) >= MAX_REVIEWS_PER_MOVIE else x)


With a max reviews per movie of 30, the average number of tokens will be approximately: 819 to 945


In [ ]:
#Display what our grouped reviews now look like for Percy Jackson.

display(grouped_reviews[grouped_reviews['movie_title'] == 'Percy Jackson & the Olympians: The Lightning Thief'])

,movie_title,review_content,review_score,critics_consensus
5002,Percy Jackson & the Olympians: The Lightning T...,As a former middle school teacher of gifted li...,5.8,Though it may seem like just another Harry Pot...


#Prepare Dataset For Model


#Stop Words

In [ ]:
custom_stop_words = {
    # 🎬 Movie-related filler words
    "movie", "movies", "film", "films", "filmmaker", "filmmakers", "cinema", "cinematic",
    "screen", "script", "scene", "scenes", "story", "storytelling", "plot", "character",
    "characters", "protagonist", "antagonist", "actor", "actors", "actress", "cast",
    "role", "roles", "performance", "performances", "line", "lines", "dialogue", "monologue",
    "shot", "shots", "frame", "frames", "footage", "editing", "visuals", "soundtrack",
    "score", "sound", "music", "direction", "director", "genre",

    # 🧠 Generic descriptors / review buzzwords
    "good", "bad", "great", "terrible", "amazing", "awful", "nice", "interesting",
    "boring", "exciting", "slow", "fast", "emotional", "intense", "strong", "weak",
    "predictable", "realistic", "unrealistic", "classic", "modern", "typical",
    "unique", "original", "complex", "simple",

    # ⏱️ Temporal / non-descriptive words
    "today", "yesterday", "now", "then", "while", "always", "never", "sometimes",
    "early", "late", "moment", "moments", "time", "times", "before", "after",
    "start", "end", "middle", "scene", "point", "part", "bit", "everything", "nothing",

    # 👥 Audience/experience terms
    "audience", "viewers", "people", "person", "someone", "anyone", "everyone",
    "nobody", "somebody", "watcher", "watching", "seen", "saw", "goers", "crowd",
    "experience", "feel", "feeling", "felt", "reactions",

    # 🚫 General filler / function words
    "just", "like", "really", "very", "quite", "well", "maybe", "actually", "however",
    "though", "although", "because", "since", "until", "yet", "still", "already",
    "some", "most", "much", "many", "few", "each", "every", "any", "all", "none"
}

In [ ]:
nltk.download('stopwords')  # Download stopwords list

# Load stop words
stop_words = set(stopwords.words("english"))

stop_words.update(custom_stop_words)

def filter_stop_words(words):
    # Filter out stop words from the list of words
    return [word for word in words if word not in stop_words]

def split_words(text):
    # Lowercase, tokenize, and remove stopwords
    words = re.findall(r"\b\w+\b", text.lower())
    return [word for word in words if word not in stop_words]

# Label words using semantic similarity from word embeddings
def label_words_with_embeddings(text, summary, threshold=0.7):
    review_words = split_words(text)
    summary_words = split_words(summary)

    if not review_words or not summary_words:
        return [], []

    # Get word embeddings (batch encode)
    review_embs = embedding_model.encode(review_words)
    summary_embs = embedding_model.encode(summary_words)

    labels = []
    for word_emb in review_embs:
        sims = cosine_similarity([word_emb], summary_embs)[0]
        max_sim = max(sims)
        labels.append(1 if max_sim >= threshold else 0)

    return review_words, labels


['watched']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#MiniLM Embedding

In [ ]:
# Sample a percentage of the grouped movies
SAMPLE_FRAC = 0.10
sampled_reviews = grouped_reviews.sample(frac=SAMPLE_FRAC, random_state=2025).reset_index(drop=True)

# Load sentence transformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare storage for word-level data
word_data = {
    "word": [],
    "label": [],
    "movie_title": [],
    "full_text": [],
    "summary": []
}

# Apply embedding-based word labeling
for _, row in tqdm(sampled_reviews.iterrows(), total=len(sampled_reviews)):
    text, summary, title = row["review_content"], row["critics_consensus"], row["movie_title"]
    if pd.isna(text) or pd.isna(summary):
        continue

    words, labels = label_words_with_embeddings(text, summary, threshold=0.6)
    word_data["word"].extend(words)
    word_data["label"].extend(labels)
    word_data["movie_title"].extend([title] * len(words))
    word_data["full_text"].extend([text] * len(words))
    word_data["summary"].extend([summary] * len(words))


100%|██████████| 891/891 [02:51<00:00,  5.20it/s]


In [ ]:
# Create DataFrame
df_classification = pd.DataFrame(word_data)
print(df_classification.head())
print(df_classification.shape)

       word  label    movie_title  \
0      stop      0  Arizona Dream   
1  possibly      0  Arizona Dream   
2      idea      0  Arizona Dream   
3     going      0  Arizona Dream   
4    happen      0  Arizona Dream   

                                           full_text  \
0  You can't stop watching because nobody in the ...   
1  You can't stop watching because nobody in the ...   
2  You can't stop watching because nobody in the ...   
3  You can't stop watching because nobody in the ...   
4  You can't stop watching because nobody in the ...   

                                             summary  
0  Inscrutably strange, yet undeniably compelling...  
1  Inscrutably strange, yet undeniably compelling...  
2  Inscrutably strange, yet undeniably compelling...  
3  Inscrutably strange, yet undeniably compelling...  
4  Inscrutably strange, yet undeniably compelling...  
(246078, 5)


In [ ]:
# Ensure label is int
df_classification["label"] = df_classification["label"].astype(int)

# Split into train and test
train_df, test_df = train_test_split(df_classification, test_size=0.2, random_state=2025)

# Convert to Hugging Face Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "validation": Dataset.from_pandas(test_df.reset_index(drop=True))
})


#Top Word Extraction

In [ ]:
def generate_top_words(text, embedding_model, stop_words=None, top_n=15, device='cuda'):
    # Split the review text into words
    words = split_words(text)

    # Remove stopwords if specified
    if stop_words:
        words = filter_stop_words(words)

    # Get word embeddings
    embeddings = embedding_model.encode(words, convert_to_tensor=True)
    embeddings = embeddings.to(device)
    embeddings_cpu = embeddings.cpu().detach().numpy()

    # Calculate similarity
    summary_emb = embedding_model.encode(words, convert_to_tensor=True).to(device).cpu().detach().numpy()
    sims = cosine_similarity(embeddings_cpu, summary_emb)

    # Sort word-similarity pairs
    word_sim_pairs = list(zip(words, sims.flatten()))
    sorted_word_sim_pairs = sorted(word_sim_pairs, key=lambda x: x[1], reverse=True)

    # Ensure unique top words
    seen = set()
    unique_top_words = []
    for word, _ in sorted_word_sim_pairs:
        word_lower = word.lower()
        if word_lower not in seen:
            seen.add(word_lower)
            unique_top_words.append(word)
        if len(unique_top_words) == top_n:
            break

    return " ".join(unique_top_words)

In [ ]:
def evaluate_summary_with_consensus(top_words_summary, critics_consensus, embedding_model, device='cuda'):
    # Encode the top words summary and the critics consensus into embeddings
    top_words_emb = embedding_model.encode(top_words_summary.split(), convert_to_tensor=True).to(device)
    consensus_emb = embedding_model.encode(critics_consensus.split(), convert_to_tensor=True).to(device)

    # Move embeddings to CPU before calculating cosine similarity
    top_words_emb = top_words_emb.cpu() # Move to CPU
    consensus_emb = consensus_emb.cpu() # Move to CPU

    # Calculate cosine similarity between the top words summary and the critics consensus
    sim = cosine_similarity(top_words_emb.detach().numpy(), consensus_emb.detach().numpy()) # Convert to NumPy arrays

    # Return the average cosine similarity score as an evaluation metric
    avg_similarity = sim.mean()  # Average similarity across all words

    return avg_similarity

In [ ]:
# Generating Top 15 Words Summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

movie_title = "Interstellar"
row = grouped_reviews[grouped_reviews["movie_title"] == movie_title].iloc[0]
review_text = row["review_content"]
critics_consensus = row["critics_consensus"]

top_words_summary = generate_top_words(
    text=review_text,
    embedding_model=embedding_model,
    stop_words=stop_words,
    top_n=15,
    device=device
)

print(f"Top 15 Words For Movie {movie_title}: {top_words_summary}")


Top 15 Words For Movie Interstellar: graceful vigorously bravery sleek inspired painfully soulful dynamic comes flaws masterpiece awkward masterful unsatisfying enthralling


#Flan Summary Generation

In [ ]:
# Load the FLAN-T5 model and tokenizer
t5_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
t5_model = t5_model.to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def generate_summary_from_words(extracted_words, movie_title=None, max_length=50):
    # Clean up the keywords
    clean_words = list(set(w.lower() for w in extracted_words if len(w) > 2))
    if not clean_words:
        return "No meaningful keywords provided."

    # Create a prompt with the movie title if provided
    if movie_title:
        prompt = (
            f"Write a short summary for the movie titled '{movie_title}' using the following key ideas: "
            + ", ".join(clean_words) + "."
        )
    else:
        prompt = "Write a short movie description using these key ideas: " + ", ".join(clean_words) + "."

    # Encode prompt
    input_ids = t5_tokenizer.encode(prompt, return_tensors="pt").to(t5_model.device)

    # Generate summary
    output_ids = t5_model.generate(
        input_ids,
        max_length=max_length,
        num_beams=5,
        do_sample=True,
        no_repeat_ngram_size=2,
        top_k=25,
        top_p=0.75,
        temperature=0.8,
        early_stopping=True
    )

    return t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
# Generate and print summary
summary = generate_summary_from_words(top_words_summary.split(), grouped_reviews[grouped_reviews['movie_title'] == 'Interstellar']['movie_title'].iloc[0])
print("Generated Summary:", summary)

Generated Summary: enthralling, masterful, sleek, dynamic, bravery, and unsettling


In [ ]:
print(grouped_reviews[grouped_reviews['movie_title'] == 'Interstellar']['critics_consensus'].iloc[0])

Interstellar represents more of the thrilling, thought-provoking, and visually resplendent filmmaking moviegoers have come to expect from writer-director Christopher Nolan, even if its intellectual reach somewhat exceeds its grasp.


# Results & Similarity Score

In [ ]:
#Generated summaries on validation set and calculate similarity score
generated_summaries = []
actual_summaries = []
similarity_scores = []
top_words_list = []

sample_size = 1000
validation_list = list(dataset['validation'])
sampled_validation = random.sample(validation_list, sample_size)

for example in tqdm(sampled_validation):
    review_text = example['full_text']
    critics_consensus = example['summary']
    movie_title = example['movie_title']

    top_words = generate_top_words(
        text=review_text,
        embedding_model=embedding_model,
        stop_words=stop_words,
        top_n=15,
        device=device
    ).split()

    generated_summary = generate_summary_from_words(top_words, movie_title)

    avg_similarity = evaluate_summary_with_consensus(
        top_words_summary=generated_summary,
        critics_consensus=critics_consensus,
        embedding_model=embedding_model,
        device=device
    )

    generated_summaries.append(generated_summary)
    actual_summaries.append(critics_consensus)
    similarity_scores.append(avg_similarity)
    top_words_list.append(top_words)


100%|██████████| 1000/1000 [30:49<00:00,  1.85s/it]


In [ ]:
#Save relevant columns from generation

results = []

for example, top_words, generated_summary, similarity in zip(sampled_validation, top_words_list, generated_summaries, similarity_scores):
    results.append({
        "movie_title": example["movie_title"],
        "original_review": example["full_text"],
        "top_words": top_words,
        "critics_consensus": example["summary"],
        "generated_summary": generated_summary,
        "similarity_score": similarity
    })

df_results = pd.DataFrame(results)
display(df_results.head())


,movie_title,original_review,top_words,critics_consensus,generated_summary,similarity_score
0,Year of the Fish,Written and directed by Reginald Harkema with ...,"[written, writer, told, directed, applied, mak...",This modernized and rotoscope-animated update ...,year of the fish tells the story of a fisherma...,0.305735
1,Royal Wedding,Mostly I sat amazed at Astaire's dancing. Most...,"[mostly, including, barely, considerably, etc,...","Vintage MGM musical stuff, characterized by St...","royal wedding is a mildly entertaining film , ...",0.271354
2,Cherish,"It's best to avoid imprisonment with the dull,...","[best, better, last, perfectly, cute, third, h...",The different genres in Cherish don't mesh wel...,a cutesy romantic comedy that keeps you guessi...,0.288763
3,Le samouraï,"Alain Delon, who was attacked by some narrow-m...","[alain, frenchman, french, pierre, le, portugu...",Le Samouraï makes the most of its spare aesthe...,samoura is a portuguese-french action thriller...,0.287008
4,Disney's A Christmas Carol,"... the heart of the story is there, and Zemec...","[heart, soul, heartfelt, emotionally, feels, s...",Robert Zemeckis' 3-D animated take on the Dick...,"uplifting and heartfelt , disney 's a christma...",0.289388


In [ ]:
#Save results to csv

from google.colab import files
df_results.to_csv('df_results_10%.csv', encoding = 'utf-8-sig')
files.download('df_results_10%.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Citations

Help from ChatGPT for key functions such as review score normalization, embedding words, and evaluating co-sine similarity.